### 三、动手实现 softmax 回归

1. 动手从 0 实现 softmax 回归（只借助于 Tensor 和 Numpy 相关的库）在 Fashion-MNIST 数据集三进行训练和测试。

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

mnist_train = torchvision.datasets.FashionMNIST(root='~/Code/Datasets/FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Code/Datasets/FashionMNIST', train=False, download=True, transform=transforms.ToTensor())

In [ ]:
print(len(mnist_train))
print(len(mnist_test))

In [ ]:
import d2l_pytorch as d2l

X, y = [], []

for i in range(10):
    X.append(mnist_train[i][0])
    y.append(mnist_train[i][1])

d2l.show_fashion_mnist(X, d2l.get_fashion_mnist_labels(y))

In [ ]:
batch_size=256
num_workers = 4
train_iter = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
num_inputs=784
num_outputs=10

W = torch.tensor(np.random.normal(0, 0.01, size=(num_inputs, num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)


In [ ]:
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

In [ ]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True)
    # print(partition.size())
    return X_exp / partition

In [ ]:
def net(X):
    return softmax(torch.mm(X.view(-1, num_inputs), W) + b)

In [ ]:
def cross_entropy(y_hat, y):
    return -torch.log(y_hat.gather(1, y.view(-1, 1)))

In [ ]:
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])
y_hat.gather(dim=1, index=y.view(-1, 1))

In [ ]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()

In [ ]:
print(accuracy(y_hat, y))

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [ ]:
print(evaluate_accuracy(test_iter, net))

In [ ]:
def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size

In [ ]:
import d2l_pytorch as d2l

num_epochs, lr = 5, 0.1
def train_service(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()

            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()

            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step()  # “softmax回归的简洁实现”一节将用到


            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [ ]:
train_service(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)

2. 利用 torch.nn 实现 softmax 回归在 Fashion_MNIST 数据集上进行训练和测试

In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.nn import init
import numpy as np
import d2l_pytorch as d2l

In [13]:
batch_size=256
mnist_train = torchvision.datasets.FashionMNIST(root='~/Code/Datasets/FashionMNIST', train=True, download=True,
                                                transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Code/Datasets/FashionMNIST', train=False, download=True,
                                               transform=transforms.ToTensor())
num_workers = 4
train_iter = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)